# EfficientNet-Lite4 - Transferred Learning

---
Link to TensorFlow Hub
https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import pathlib


In [ ]:
num_classes = 6
# Load the EfficientNet-Lite model from TensorFlow Hub
module_url = "https://tfhub.dev/tensorflow/efficientnet/lite4/classification/2"
model = Sequential([
    hub.KerasLayer(module_url, trainable=False, input_shape=(260, 260, 3)),
    Dense(num_classes, activation='softmax')
])

In [ ]:
data_dir = '../SFSU_Art'
data_dir = pathlib.Path(data_dir)
#print(data_dir)

In [ ]:
batch_size = 32
img_height = 260
img_width = 260

# Training data  
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_classnames = train_ds.class_names
#with open('./models/label.txt', 'w') as f:
#    f.writelines(str(item) + '\n' for item in class_names)



# Fixing the number output shape

---


In [ ]:
# get the configuration of the final layer
output_layer_config = output_layer.get_config()

# print the configuration
print(output_layer_config)

# get the output layer of the model
output_layer = model.layers[-1]


# create a new input layer with 1 unit
new_input_layer = Dense(1)(output_layer.output)

# create a new output layer with the desired number of units and activation function
new_output_layer = Dense(6, activation='softmax')(output_layer.output)

# create a new model with the same input and all layers except the original output layer
new_model = keras.Model(inputs=model.input, outputs=new_output_layer, name='new_model')


In [ ]:
# Compile the model with a categorical crossentropy loss function and an Adam optimizer
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
new_model.summary()


In [ ]:
# print the configuration of the previous layer
previous_layer_config = model.layers[-2].get_config()
print(previous_layer_config)


In [ ]:
# Train the model on a new dataset
epochs = 15
history = new_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)